In [31]:
%cd /Users/maayanlab/Documents/projects/human-melanoma-signature
import sys
sys.path.append('pipeline/scripts/')
import pandas as pd
import h5py
import numpy as np
import json, operator, math
from geode_jupies import *
import sklearn.metrics.pairwise as smp
from scipy.spatial.distance import cosine
import seaborn as sns; sns.set(color_codes=True)

/Users/maayanlab/Documents/projects/human-melanoma-signature


In [2]:
%pwd

'/Users/maayanlab/Documents/projects/human-melanoma-signature'

In [3]:
from Melanoma import *

# Code Development for Melanoma Pipeline
## Define input and output files

In [16]:
# infiles
infile = 's2-signatures.dir/melanoma-signatures.txt'

In [4]:
# #infiles
# sig = 's2-signatures.dir/melanoma-signatures.txt'

In [5]:
# #infiles
# cpm = 's1-expression_data.dir/melanoma-cpm.txt'
# matches = 's2-expression_data.dir/melanoma-idmatches.txt'

In [7]:
# # Infile & outfile
# infiles = ['s1-expression_data.dir/melanoma-cpm.txt']
# outfile = 's1-expression_data.dir/melanoma-IDstatus.txt'

In [8]:
# # Infile & outfile
# infiles = ['rawdata/human_matrix.h5', 'rawdata/m2.csv']
# outfile = 's1-expression_data.dir/melanoma-cpm.txt'

### Test Code

In [26]:
# Import infile table
signature_df = pd.read_table(infile, index_col='gene')#.T
signature_df.head()

,1,11,15_a,15_b,15_c,17_a,17_b,17_c,18_a,18_b,...,19_b,19_c,19_d,2_a,2_b,2_c,3,5_a,5_b,5_c
gene,,,,,,,,,,,,,,,,,,,,,
A1BG,0.000222,-0.000741,-0.000528,-0.000293,0.000081,-0.000679,-0.000913,-0.000520,0.000217,-0.000439,...,-0.001003,-0.000709,-0.000192,-0.000019,-0.000432,-0.000729,-0.000202,0.001092,0.000437,0.000112
A2M,0.006467,-0.000725,-0.002951,-0.003329,-0.002101,-0.024287,-0.024964,-0.002382,-0.000219,-0.001720,...,0.000173,0.000145,-0.000058,-0.008256,0.273626,0.135737,0.041137,0.000944,-0.019554,-0.000980
A2ML1,-0.000124,-0.000725,-0.000793,-0.000344,0.000104,-0.000761,-0.000908,-0.000830,0.000205,-0.000298,...,0.000173,-0.000058,0.000139,0.000249,-0.000923,-0.001315,-0.000556,0.001044,0.000681,0.000015
A2MP1,-0.000104,-0.000750,-0.000818,-0.000365,0.000101,-0.000775,-0.000919,-0.000840,0.000208,-0.000287,...,0.000162,-0.000056,0.000130,0.000238,-0.000903,-0.001314,-0.000528,0.001051,0.000619,0.000016
A4GALT,0.000729,-0.000740,-0.000195,-0.001962,-0.000815,-0.000589,-0.000910,-0.000797,0.000213,-0.000504,...,0.000122,-0.000156,0.000164,0.000202,-0.000940,-0.001320,0.002802,0.001034,0.000665,-0.000391


In [27]:
# Transpose dataframe so index is n_samples, which is required for pair-wise distance function
signature_df_t = signature_df.T

In [28]:
signature_df_t

gene,A1BG,A2M,A2ML1,A2MP1,A4GALT,AAAS,AACS,AACSP1,AADACP1,AADAT,...,ZWILCH,ZWINT,ZXDA,ZXDB,ZXDC,ZYG11A,ZYG11B,ZYX,ZZEF1,ZZZ3
1,0.000222,0.006467,-0.000124,-0.000104,0.000729,-0.000029,0.000268,-0.000091,-0.000111,-0.000467,...,-0.000740,-0.001510,0.000112,0.000180,0.000524,-0.000110,0.000962,0.006551,0.001210,-0.002782
11,-0.000741,-0.000725,-0.000725,-0.000750,-0.000740,-0.000671,0.005065,-0.000709,-0.000780,-0.000602,...,-0.001459,0.001186,-0.000609,-0.000490,-0.000319,-0.000718,0.000418,-0.000092,0.001766,-0.001441
15_a,-0.000528,-0.002951,-0.000793,-0.000818,-0.000195,0.000764,-0.000697,-0.000794,-0.000779,-0.000966,...,0.000337,0.000990,-0.000404,-0.000371,-0.001036,-0.000829,0.000289,-0.005603,0.001405,0.001078
15_b,-0.000293,-0.003329,-0.000344,-0.000365,-0.001962,0.000602,-0.000205,-0.000353,-0.000362,-0.000498,...,0.001658,0.002164,-0.000190,-0.000013,-0.000746,-0.000317,0.001126,-0.007491,0.000715,0.002051
15_c,0.000081,-0.002101,0.000104,0.000101,-0.000815,-0.001139,-0.000929,0.000102,0.000115,0.000011,...,-0.000265,-0.001082,0.000258,0.000372,0.000821,0.000090,-0.000573,0.012119,-0.000742,0.001349
17_a,-0.000679,-0.024287,-0.000761,-0.000775,-0.000589,0.000240,-0.000864,-0.000758,-0.000758,-0.000630,...,0.000568,0.002400,-0.000743,-0.000649,-0.000292,-0.000469,0.000003,0.000430,-0.000941,0.001610
17_b,-0.000913,-0.024964,-0.000908,-0.000919,-0.000910,0.000240,-0.000125,-0.000902,-0.000903,0.000302,...,0.001447,0.002904,-0.000838,-0.000394,-0.000464,-0.000918,-0.000575,0.006078,-0.000594,0.002857
17_c,-0.000520,-0.002382,-0.000830,-0.000840,-0.000797,0.000656,-0.000464,-0.000758,-0.000831,-0.000789,...,0.000145,0.000052,-0.000779,-0.000604,-0.000072,-0.000846,0.000406,0.004623,-0.000349,-0.001222
18_a,0.000217,-0.000219,0.000205,0.000208,0.000213,-0.000428,-0.000521,0.000169,0.000198,0.000257,...,0.000170,-0.001709,0.000182,0.000279,-0.000007,0.000193,0.000500,-0.000979,-0.000224,0.000778
18_b,-0.000439,-0.001720,-0.000298,-0.000287,-0.000504,-0.002003,0.003434,-0.000357,-0.000299,-0.000130,...,0.001834,0.000139,-0.000350,-0.000665,0.000061,-0.000320,-0.001679,0.003622,0.001543,-0.000157


In [32]:
# Create an empty dataframe with study_index as both index and column labels
cos_distance_df = pd.DataFrame(index=signature_df_t.index, columns=signature_df_t.index)
# Compute the distance matrix from a vector array X
cos_distance = smp.pairwise_distances(signature_df_t, metric = 'cosine')

In [33]:
cos_distance

array([[0.        , 0.92597166, 0.84920912, 0.98359232, 0.98102924,
        1.26579641, 1.0880072 , 1.08283475, 0.97681895, 1.02521806,
        1.04080054, 0.94307955, 0.98924894, 0.77090293, 1.13787358,
        0.9934145 , 1.03386811, 1.0322871 , 1.06455708, 1.00449387,
        1.41639697, 1.09479   ],
       [0.92597166, 0.        , 0.84714008, 0.89891276, 1.09730119,
        0.99975815, 0.95540007, 0.98846449, 0.98408316, 0.93775953,
        1.04643288, 0.85838951, 1.11701046, 1.01671353, 1.05963527,
        1.06447959, 0.9675689 , 0.96248823, 0.99102653, 1.12203317,
        1.04791512, 1.02478702],
       [0.84920912, 0.84714008, 0.        , 0.17244681, 1.76196933,
        1.09741176, 1.06562658, 0.99048613, 0.97662416, 0.93704887,
        1.08622529, 0.7323953 , 1.22372941, 1.03434121, 1.14456426,
        1.03673953, 1.0181972 , 0.99259671, 0.90983251, 1.08529573,
        1.15993085, 1.12987081],
       [0.98359232, 0.89891276, 0.17244681, 0.        , 1.79537371,
        1.1130571

In [ ]:
	# Import infile table
	signature_df = pd.read_table(infile)
	# Transpose dataframe so index is n_samples, which is required for pair-wise distance function
	signature_df_t = signature_df.T
	# Create an empty dataframe with study_index as both index and column labels
	cos_distance_df = pd.DataFrame(index=signature_df_t.index, columns=signature_df_t.index)
	# Compute the distance matrix from a vector array X
	cos_distance = pairwise_distances(signature_df_t, metric = 'cosine')
	# Note that cosine distance is defined as 1.0 minus the cosine similarity, hence a completely different function
	# Fill in the empty df earlier with values from cos_distance array
	cos_distance_df[:]=cos_distance